In [2]:
import pandas as pd

In [3]:
consumer_price = pd.read_csv("data/consumer-price.csv")
govt_spending = pd.read_csv("data/govt-spending.csv")
expense_gdp = pd.read_csv("data/expense-gdp.csv")
gdp = pd.read_csv("data/gdp.csv")

In [4]:
print(consumer_price.info())
print(consumer_price.isnull().sum())
print(consumer_price.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8904 entries, 0 to 8903
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Entity                8904 non-null   object 
 1   Code                  8904 non-null   object 
 2   Year                  8904 non-null   int64  
 3   Consumer_price_index  8904 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 278.4+ KB
None
Entity                  0
Code                    0
Year                    0
Consumer_price_index    0
dtype: int64
        Entity Code  Year  Consumer_price_index
0  Afghanistan  AFG  2004             63.523396
1  Afghanistan  AFG  2005             71.582146
2  Afghanistan  AFG  2006             76.438705
3  Afghanistan  AFG  2007             83.074020
4  Afghanistan  AFG  2008            105.021065


In [5]:
# Drop the '935576-annotations' column
govt_spending = govt_spending.drop(columns=['935576-annotations'])

# Round it to the nearest whole number
govt_spending["Government expenditure (% of GDP)"] = (
    pd.to_numeric(govt_spending["Government expenditure (% of GDP)"], errors="coerce").round(2)
)

In [6]:
print(govt_spending.info())
print(govt_spending.isnull().sum())
print(govt_spending.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9092 entries, 0 to 9091
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Entity                             9092 non-null   object 
 1   Code                               9092 non-null   object 
 2   Year                               9092 non-null   int64  
 3   Government expenditure (% of GDP)  9092 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 284.3+ KB
None
Entity                               0
Code                                 0
Year                                 0
Government expenditure (% of GDP)    0
dtype: int64
    Entity Code  Year  Government expenditure (% of GDP)
0  Albania  ALB  1997                              30.30
1  Albania  ALB  1998                              36.75
2  Albania  ALB  1999                              37.11
3  Albania  ALB  2000                       

In [7]:
# Fill missing World_regions based on Entity
expense_gdp["World_regions"] = (
    expense_gdp.groupby("Entity")["World_regions"]
    .transform(lambda x: x.ffill().bfill())
)

# If an entity has no World_regions at all, fill it with Entity name
expense_gdp["World_regions"] = expense_gdp["World_regions"].fillna(expense_gdp["Entity"])

# Fill missing values using bfill within each entity, then replace remaining NaNs with 0
expense_gdp[["Consumption_expenditure", "GDP"]] = (
    expense_gdp.groupby("Entity")[["Consumption_expenditure", "GDP"]]
    .transform(lambda x: x.bfill())
    .fillna(0)
)

expense_gdp["Code"] = expense_gdp["Code"].fillna("N/A")

# Round float values to ensure they are whole numbers before conversion
expense_gdp["Consumption_expenditure"] = expense_gdp["Consumption_expenditure"].round(0).astype("Int64")
expense_gdp["GDP"] = expense_gdp["GDP"].round(0).astype("Int64")

# Sort the data by Entity and Year within each group
expense_gdp = expense_gdp.sort_values(by=["Entity", "Year"])

# Reset the index if needed
expense_gdp = expense_gdp.reset_index(drop=True)

In [8]:
print(expense_gdp.info())
print(expense_gdp.isnull().sum())
print(expense_gdp.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12044 entries, 0 to 12043
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Entity                   12044 non-null  object
 1   Code                     12044 non-null  object
 2   Year                     12044 non-null  int64 
 3   Consumption_expenditure  12044 non-null  Int64 
 4   GDP                      12044 non-null  Int64 
 5   World_regions            12044 non-null  object
dtypes: Int64(2), int64(1), object(3)
memory usage: 588.2+ KB
None
Entity                     0
Code                       0
Year                       0
Consumption_expenditure    0
GDP                        0
World_regions              0
dtype: int64
        Entity Code  Year  Consumption_expenditure         GDP World_regions
0  Afghanistan  AFG  2000                        0  6206547500          Asia
1  Afghanistan  AFG  2001                        0  56211476

In [9]:
# Fill missing "Code" with "N/A"
gdp["Code"].fillna("N/A", inplace=True)

In [10]:
print(gdp.info())
print(gdp.isnull().sum())
print(gdp.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11965 entries, 0 to 11964
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Entity          11965 non-null  object 
 1   Code            11965 non-null  object 
 2   Year            11965 non-null  int64  
 3   GDP_per_capita  11965 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 374.0+ KB
None
Entity            0
Code              0
Year              0
GDP_per_capita    0
dtype: int64
        Entity Code  Year  GDP_per_capita
0  Afghanistan  AFG  2000       308.31827
1  Afghanistan  AFG  2001       277.11804
2  Afghanistan  AFG  2002       338.13998
3  Afghanistan  AFG  2003       346.07162
4  Afghanistan  AFG  2004       338.63727


In [11]:
consumer_price.to_csv("cleaned/consumer_price.csv", index=False)
govt_spending.to_csv("cleaned/govt_spending.csv", index=False)
expense_gdp.to_csv("cleaned/expense-gdp.csv", index=False)
gdp.to_csv("cleaned/gdp.csv", index=False)